<h1>AUTOMATIZACIÓN CONSOLIDADO SAT PUSH EXPERIENCIA<h1>

In [1]:
import pandas as pd
import os
import glob
import re

In [2]:
mes_anterior_num = "05"
mes_anterior_str = "Mayo"

In [6]:
carpeta_excel = f'C:/Users/BBARTOLOMEE/OneDrive - Entel Peru S.A/03. Programación Diaria Comunicaciones/02. Bases Satpush (CVM)/2025/{mes_anterior_num}. {mes_anterior_str}'
archivos_excel = glob.glob(os.path.join(carpeta_excel, '*.xlsx'))

dataframes_excel = []

In [7]:
for archivo in archivos_excel:
    try:
        df = pd.read_excel(archivo)
        nombre_archivo = os.path.basename(archivo).replace('.xlsx', '')
        df['archivo'] = nombre_archivo
        dataframes_excel.append(df)
    except Exception as e:
        print(f"Error leyendo {archivo}: {e}")

df_consolidado = pd.concat(dataframes_excel, ignore_index=True)

In [ ]:
df_consolidado

<h3>BASE RUCS<h3>

In [ ]:
carpeta_rucs = f'C:/Users/BBARTOLOMEE/OneDrive - Entel Peru S.A/03. Programación Diaria Comunicaciones/02. Bases Satpush (CVM)/2025/RUC {mes_anterior_str}.xlsx'

In [ ]:
df_base_ruc = pd.read_excel(carpeta_rucs)
df_base_ruc.drop_duplicates(subset='TELEFONO')

In [11]:
df_merge = pd.merge(df_consolidado, df_base_ruc, on='TELEFONO', how='left')

In [12]:
df_merge['nombre_base'] = df_merge['archivo'].str.extract(r'_(?:.*?)_(.*)$')
df_merge['FECHA'] = pd.to_datetime(df_merge['archivo'].str[:8], format='%Y%m%d')
df_merge['FECHA'] = df_merge['FECHA'].dt.strftime('%d-%m-%Y')

In [ ]:
df_merge

In [ ]:
#df_nombre = df_merge[['nombre_base']].drop_duplicates()
#df_nombre.to_excel('base_campaña.xlsx',index = False)

<h3>BASE CAMPAÑAS<h3>

In [14]:
base_campaña = pd.read_excel('base_campaña.xlsx')
base_final_1 = pd.merge(df_merge, base_campaña, on = 'nombre_base', how='left')

In [15]:
base_final_1[['TIPO']] = 'SAT PUSH'
base_final_1[['OWNER']] = 'CVM'

In [16]:
base_final_2 = base_final_1[['FECHA','RUCCOMPANIA','TELEFONO','OWNER','Campaña','TIPO']]
base_final_2.to_excel('df_final.xlsx',index = False)